In [2]:
from neo4j import GraphDatabase

In [9]:
driver = GraphDatabase.driver('bolt://localhost:7687',auth=('neo4j','1'))

In [53]:
name = "ALA"
title = "ALA THE MOVIE"

def create_actor(tx,name):
    result = tx.run(f'CREATE (a:Actor{{name: "{name}"}}) RETURN a')
#     print(list(result))
        
def create_movie(tx,name):
    result = tx.run(f'CREATE (m:Movie{{title: "{title}"}}) RETURN m')
#     print(list(result))
     
def create_relation(tx,name,title):
    result = tx.run(f'''
            MATCH (a:Actor{{name: "{name}"}}),(m:Movie{{title: "{title}"}})
            CREATE (a) -[:ACTS_IN]-> (m)
            ''')
#     print(list(result))
    
with driver.session() as session:
    session.write_transaction(create_actor,name)
    session.write_transaction(create_movie,title)
    session.write_transaction(create_relation,name,title)

In [54]:
birthplace = 'Miasto zwane Miastem'
birthdate = '99-99-9999'

def set_actor(tx,name,birthplace,birthdate):
    result = tx.run(f'''
            MATCH (a:Actor{{name: "{name}"}})
            SET a.birthplace = "{birthplace}"
            SET a.birthdate = "{birthdate}"
            ''')
#     print(list(result))
           
with driver.session() as session:
    session.write_transaction(set_actor,name,birthplace,birthdate)

In [56]:
def count_query(tx):
    result = tx.run(f'''
            MATCH (a:Actor) -[r:ACTS_IN]->() WITH a, length(collect(r)) as c WHERE c > 6 RETURN a
            ''')
#     print(list(result))
    
with driver.session() as session:
    session.read_transaction(count_query)

In [57]:
def avg_query(tx):
    result = tx.run(f'''
                    MATCH (a:Actor) -[r:ACTS_IN]->() WITH a, length(collect(r)) as c WHERE c > 7 RETURN AVG(c)
                    ''')
#     print(list(result))

with driver.session() as session:
    session.read_transaction(avg_query)

[<Record AVG(c)=14.071654373024238>]


In [59]:
def actor_director_query(tx):
    result = tx.run(f'''
                    MATCH ()<-[r2:DIRECTED]- (a:Actor) -[r1:ACTS_IN]->() 
                    WITH a, COUNT(r1) AS c1, COUNT(r2) AS c2 
                    WHERE c1 > 5 AND c2 > 1
                    RETURN a
                    ORDER BY c1
                    ''')
#     print(list(result))

with driver.session() as session:
    session.read_transaction(actor_director_query)

In [62]:
login = 'maheshksp'
def friend_rate_query(tx,login):
    result = tx.run(f'''
                    MATCH (u:User{{login: "{login}"}}) -[:FRIEND] -> (f:User) -[r:RATED] -> (m:Movie) 
                    WHERE r.stars > 2 
                    RETURN f,m.title,r.stars
                    ''')
#     print(list(result))
    
with driver.session() as session:
    session.read_transaction(friend_rate_query,login)

## TODO
10. Zapytanie o ścieżki między wybranymi aktorami (np.2), w ścieżkach mają nie znajdować się filmy (funkcja filter(), [x IN xs WHERE predicate | extraction])
11. Porównać czas wykonania zapytania o wybranego aktora bez oraz z indeksem w bazie nałożonym na atrybut name (DROP INDEX i CREATE INDEX oraz użyć komendy EXPLAIN lub PROFILE), dokonać porównania dla zapytania shortestPath pomiędzy dwoma wybranymi aktorami.